In [9]:
import requests
import json
import time
import pymongo
import pandas as pd

In [60]:
usa = pd.read_csv('usa.csv')
ZIP = ["36104","99801","85001","72201","95814","80202","06103"
    ,"19901","32301","30303","96813","83702","62701","46225","50309"
    ,"66603","40601","70802","04330","21401","02201","48933","55102"
    ,"39205","65101","59623","68502","89701","03301","08608","87501"
    ,"12207","27601","58501","43215","73102","97301","17101","02903"
    ,"29217","57501","37219","78701","84111","05602","23219","98507"
    ,"25301","53703","82001"]
STATE = usa.head(50)['State'].values

LAT = ["32.377716","58.301598","33.448143","34.746613","38.576668","39.739227",
        "41.764046","39.157307","21.307442","30.438118","33.749027","43.617775",
        "39.798363","39.768623","41.591087","39.048191","38.186722","30.457069",
        "44.307167","38.978764","42.358162","42.733635","44.955097","32.303848",
        "38.579201","46.585709","40.808075","39.163914","43.206898","40.220596",
        "35.68224","35.78043","46.82085","42.652843","39.961346","35.492207",
        "44.938461","40.264378","41.830914","34.000343","44.367031","36.16581",
        "30.27467","40.777477","44.262436","37.538857","47.035805","38.336246",
        "43.074684","41.140259"]
LON = ["-86.300568","-134.420212","-112.096962","-92.288986","-121.493629","-104.984856",
        "-72.682198","-75.519722","-157.857376","-84.281296","-84.388229","-116.199722",
        "-89.654961","-86.162643","-93.603729","-95.677956","-84.875374","-91.187393",
        "-69.781693","-76.490936","-71.063698","-84.555328","-93.102211","-90.182106",
        "-92.172935","-112.018417","-96.699654","-119.766121","-71.537994","-74.769913",
        "-105.939728","-78.639099","-100.783318","-73.757874","-82.999069","-97.503342",
        "-123.030403","-76.883598","-71.414963","-81.033211","-100.346405","-86.784241",
        "-97.740349","-111.888237","-72.580536","-77.43364","-122.905014","-81.612328",
        "-89.384445","-104.820236"]

In [62]:
len(ZIP)

50

Get current weather condition.

In [63]:
# https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?zip="
API_KEY = "4bef6b99cccc82e075c4404ee09824f5"
# 3090ecca0f9472002c4fb40568723c37
data = []
for zipcode in ZIP:
    URL = BASE_URL + zipcode + "&units=imperial&appid=" + API_KEY
    response = requests.get(URL)
    data.append(response.json())

In [64]:
client = pymongo.MongoClient()
client.list_database_names()
database = client['test']
collection = database['weather']
collection = database['current']
collection.drop()
collection = database['current']
for jsn in data:
    collection.insert_one(jsn)

In [65]:
data[20]

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 70.7,
  'feels_like': 70.66,
  'temp_min': 65.05,
  'temp_max': 75.16,
  'pressure': 1018,
  'humidity': 68},
 'visibility': 10000,
 'wind': {'speed': 1.99, 'deg': 0, 'gust': 0},
 'clouds': {'all': 75},
 'dt': 1634073609,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1634036020,
  'sunset': 1634076438},
 'timezone': -14400,
 'id': 0,
 'name': 'Boston',
 'cod': 200,
 '_id': ObjectId('6165fc1b0d582e2632521b1b')}

Get 4-days weather condition hourly.

In [31]:
# https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&appid={API key}
BASE_URL = "https://api.openweathermap.org/data/2.5/onecall?lat="
API_KEY = "3090ecca0f9472002c4fb40568723c37"
data = []
for idx in range(len(LAT)):
    URL = BASE_URL + LAT[idx] +"&lon=" + LON[idx] + "&exclude=current,minutely,hourly,alerts&units=imperial&appid=" + API_KEY
    response = requests.get(URL)
    data.append(response.json())

In [32]:
client = pymongo.MongoClient()
client.list_database_names()
database = client['test']
collection = database['forecast']
for jsn in data:
    collection.insert_one(jsn)

Get historical data 5 days.

In [48]:
# https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={API key}
BASE_URL = "https://api.openweathermap.org/data/2.5/onecall/timemachine?lat="
API_KEY = "3090ecca0f9472002c4fb40568723c37"
data = []
dt_sec = 86400
for idx in range(len(LAT)):
    dt = int(time.time()) - (5 * 86400)
    for i in range(5):
        URL = BASE_URL + LAT[idx] +"&lon=" + LON[idx] + "&dt=" + str(dt) + "&units=imperial&appid=" + API_KEY
        dt += dt_sec
        response = requests.get(URL)
        data.append(response.json())
    

In [49]:
client = pymongo.MongoClient()
client.list_database_names()
database = client['test']
collection = database['history']
# collection.drop()
# collection = database['history']
for jsn in data:
    collection.insert_one(jsn)

In [43]:
cursor = collection.find({},{'lat':1,'lon':1,'current':1})
entity = list(cursor)

2019 - 2021 historical data.

In [58]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
data_path = "data/combined.csv"

In [69]:
df = spark.read.csv(data_path, header='true', inferSchema='true')
df.createOrReplaceTempView("data")
df.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- AWND: string (nullable = true)
 |-- TAVG: string (nullable = true)
 |-- TMAX: string (nullable = true)
 |-- TMIN: string (nullable = true)
 |-- WT01: string (nullable = true)
 |-- WT02: string (nullable = true)
 |-- WT03: string (nullable = true)
 |-- WT04: string (nullable = true)
 |-- WT05: string (nullable = true)
 |-- WT06: string (nullable = true)
 |-- WT07: string (nullable = true)
 |-- WT08: string (nullable = true)
 |-- WT09: string (nullable = true)
 |-- WT11: string (nullable = true)



In [82]:
spark.sql('SELECT * FROM data WHERE (LATITUDE BETWEEN 30 AND 34) AND (LONGITUDE BETWEEN -88 AND -84)').show()

+-----------+--------------------+--------+---------+---------+----------+-----+----+----+----+-----+-----+-----+----+----+----+----+-----+----+----+
|    STATION|                NAME|LATITUDE|LONGITUDE|ELEVATION|      DATE| AWND|TAVG|TMAX|TMIN| WT01| WT02| WT03|WT04|WT05|WT06|WT07| WT08|WT09|WT11|
+-----------+--------------------+--------+---------+---------+----------+-----+----+----+----+-----+-----+-----+----+----+----+----+-----+----+----+
|USW00013895|MONTGOMERY AIRPOR...| 32.2997| -86.4075|     61.6|2021-01-01| 8.50|  69|  72|  58|    1| null|    1|null|null|null|null| null|null|null|
|USW00013895|MONTGOMERY AIRPOR...| 32.2997| -86.4075|     61.6|2021-01-02| 5.82|  58|  63|  47| null| null| null|null|null|null|null| null|null|null|
|USW00013895|MONTGOMERY AIRPOR...| 32.2997| -86.4075|     61.6|2021-01-03| 7.61|  49|  56|  36| null| null| null|null|null|null|null| null|null|null|
|USW00013895|MONTGOMERY AIRPOR...| 32.2997| -86.4075|     61.6|2021-01-04| 2.91|  45|  63|  31|    1